<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/5_1_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация CatBoostClassifier с использованием GPU (CUDA 12.5) для классификации узлов на датасете Bitcoin-OTC

Задача — предсказать "уровень доверия" пользователей

Признаки узлов/рёбер - синтетические

Небходимо выбрать среду выполнения с GPU: Среда выполнения → Сменить среду выполнения → Графический процессор T4

## 1. Установка зависимостей

Требования:
- CatBoost ≥1.2.2
- PyTorch Geometric для загрузки данных


* CatBoost автоматически использует доступные GPU через CUDA, явная установка CUDA 12.5 не требуется

In [1]:
install = True
if install:
  !pip install -q catboost
  !pip install -q torch-geometric

In [2]:
from google.colab import output
output.disable_custom_widget_manager()

## 2. Импорт библиотек

In [3]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from torch_geometric.datasets import BitcoinOTC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 3. Загрузка данных

In [4]:
dataset = BitcoinOTC(root='/tmp/BitcoinOTC')
data = dataset[0]

## 4. Создание признаков узлов с учётом рёбер

In [5]:
def create_node_features(data):
    num_nodes = data.num_nodes
    features = []

    # Степень узлов
    degrees = np.zeros(num_nodes)
    for src in data.edge_index[0].unique().cpu().numpy():
        degrees[src] = (data.edge_index[0].cpu().numpy() == src).sum()

    # Статистики по рёбрам
    edge_attrs = [[] for _ in range(num_nodes)]
    for i in range(data.edge_index.size(1)):
        src = data.edge_index[0, i].item()
        attr = data.edge_attr[i].item()
        edge_attrs[src].append(attr)

    # Создание фичей
    for node_id in range(num_nodes):
        node_feats = [
            degrees[node_id],  # Степень узла
            np.mean(edge_attrs[node_id]) if edge_attrs[node_id] else 0,  # Среднее значение рёбер
            np.max(edge_attrs[node_id]) if edge_attrs[node_id] else 0,   # Максимум
            np.min(edge_attrs[node_id]) if edge_attrs[node_id] else 0,   # Минимум
            len(edge_attrs[node_id])                                      # Количество рёбер
        ]
        features.append(node_feats)

    return np.array(features)

## 5. Подготовка данных

In [6]:
X = create_node_features(data)
X

array([[2. , 2.5, 4. , 1. , 2. ],
       [3. , 5. , 5. , 5. , 3. ],
       [0. , 0. , 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]])

In [7]:
y = np.zeros(data.num_nodes)

# Создание меток (3 класса)
q1 = np.quantile(X[:, 0], 0.5)
q2 = np.quantile(X[:, 0], 0.999)
y[X[:, 0] > q2] = 2
y[(X[:, 0] > q1) & (X[:, 0] <= q2)] = 1

## 6. Разделение данных

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

## 7. Создание CatBoost модели с GPU

In [9]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    task_type='GPU',  # Включение GPU
    devices='0:1',    # Использование первого GPU
    verbose=100,
    early_stopping_rounds=50
)

## 8. Обучение

In [10]:
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9095125	test: 0.9095340	best: 0.9095340 (0)	total: 55.9ms	remaining: 55.8s
100:	learn: 0.0003977	test: 0.0004117	best: 0.0004117 (100)	total: 1.38s	remaining: 12.3s
200:	learn: 0.0001106	test: 0.0001164	best: 0.0001164 (200)	total: 6.06s	remaining: 24.1s
300:	learn: 0.0000637	test: 0.0000674	best: 0.0000674 (300)	total: 10.1s	remaining: 23.5s
400:	learn: 0.0000444	test: 0.0000471	best: 0.0000471 (400)	total: 13.5s	remaining: 20.2s
500:	learn: 0.0000340	test: 0.0000360	best: 0.0000360 (500)	total: 18.7s	remaining: 18.6s
600:	learn: 0.0000276	test: 0.0000293	best: 0.0000293 (600)	total: 19.6s	remaining: 13s
700:	learn: 0.0000231	test: 0.0000245	best: 0.0000245 (700)	total: 20.3s	remaining: 8.65s
800:	learn: 0.0000199	test: 0.0000211	best: 0.0000211 (800)	total: 21s	remaining: 5.23s
900:	learn: 0.0000174	test: 0.0000185	best: 0.0000185 (900)	total: 21.8s	remaining: 2.39s
999:	learn: 0.0000157	test: 0.0000167	best: 0.0000167 (999)	total: 22.5s	remaining: 0us
bestTest = 1.666172

## 9. Оценка

In [11]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1198
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1

    accuracy                           1.00      1201
   macro avg       1.00      1.00      1.00      1201
weighted avg       1.00      1.00      1.00      1201



## 10. Важность признаков

In [12]:
feature_names = ['Degree', 'EdgeMean', 'EdgeMax', 'EdgeMin', 'EdgeCount']
print("\nFeature Importance:")
for name, score in zip(feature_names, model.get_feature_importance()):
    print(f"{name}: {score:.2f}")


Feature Importance:
Degree: 65.10
EdgeMean: 7.08
EdgeMax: 0.04
EdgeMin: 1.49
EdgeCount: 26.29


## 11. Сохранение модели

In [13]:
#model.save_model('catboost_model.cbm')

## 12. Загрузка модели

In [14]:
#loaded_model = CatBoostClassifier()
#loaded_model.load_model('catboost_model.cbm')

In [15]:
#y_pred = loaded_model.predict(X_test)
#print(classification_report(y_test, y_pred, labels=[0, 1, 2]))